In [2]:
import importlib
import pickle
import pandas as pd
import torch
import sys
sys.path.append("/home/jarobyte/guemes/lib")
from pytorch_decoding import seq2seq
import ocr_correction
from timeit import default_timer as t
from tqdm.notebook import tqdm
from ocr_correction import *

In [4]:
importlib.reload(ocr_correction)
language = "de"
model_id = "22694260_9"

scratch = f"/home/jarobyte/scratch/guemes/icdar/{language}/"

print(f"Evaluating {language}...")
data = pd.read_pickle(f"/home/jarobyte/scratch/guemes/icdar/{language}/data/test.pkl")
device = torch.device("cuda")
model = seq2seq.load_architecture(scratch + f"baseline/models/{model_id}.arch")
model.to(device)
model.eval()
model.load_state_dict(torch.load(scratch + f"baseline/checkpoints/{model_id}.pt"))
with open(scratch + "data/vocabulary.pkl", "rb") as file:
    vocabulary = pickle.load(file)
len(vocabulary)

Evaluating de...
Model: Transformer
Tokens in the input vocabulary: 337
Tokens in the output vocabulary: 337
Max sequence length: 110
Embedding dimension: 128
Feedforward dimension: 512
Encoder layers: 4
Decoder layers: 4
Attention heads: 8
Activation: relu
Dropout: 0.3
Trainable parameters: 1,995,729



334

In [ ]:
raw = data.ocr_to_input
gs = data.gs_aligned
document_progress_bar = 0
window_size = 50
metrics = []
old = levenshtein(reference = gs, hypothesis = raw).cer.mean()

In [27]:
start = t()
corrections = [correct_by_disjoint_window(s, 
                                          model, 
                                          vocabulary, 
                                          document_progress_bar = 0, 
                                          window_size = window_size) 
               for s in tqdm(raw)]
metrics.append({"window":"disjoint", 
                "decoding":
                "greedy",
                "window_size":window_size * 2,
                "inference_seconds":t() - start,
                "cer_before":old,
                "cer_after":levenshtein(gs, corrections).cer.mean()})

  0%|          | 0/592 [00:00<?, ?it/s]

In [28]:
start = t()
corrections = [correct_by_disjoint_window(s, 
                                          model, 
                                          vocabulary, 
                                          document_progress_bar = document_progress_bar, 
                                          window_size = window_size * 2) 
               for s in raw]
metrics.append({"window":"disjoint", 
                "decoding":"greedy",
                "window_size":window_size,
                "inference_seconds":t() - start,
                "cer_before":old,
                "cer_after":levenshtein(gs, corrections).cer.mean()})

In [30]:
start = t()
corrections = [correct_by_disjoint_window(s, 
                                          model, 
                                          vocabulary, 
                                          decoding_method = "beam_search", 
                                          document_progress_bar = document_progress_bar, 
                                          window_size = window_size) 
               for s in tqdm(raw)]
metrics.append({"window":"disjoint", 
                "decoding":"beam", 
                "window_size":window_size * 2,
                "inference_seconds":t() - start,
                "cer_before":old,
                "cer_after":levenshtein(gs, corrections).cer.mean()})
start = t()
corrections = [correct_by_disjoint_window(s, 
                                          model, 
                                          vocabulary, 
                                          decoding_method = "beam_search", 
                                          document_progress_bar = document_progress_bar, 
                                          window_size = window_size * 2) 
               for s in tqdm(raw)]
metrics.append({"window":"disjoint", 
                "decoding":"beam", 
                "window_size":window_size,
                "inference_seconds":t() - start,
                "cer_before":old,
                "cer_after":levenshtein(gs, corrections).cer.mean()})

  0%|          | 0/592 [00:00<?, ?it/s]

  0%|          | 0/592 [00:00<?, ?it/s]

In [34]:
start = t()
corrections = [correct_by_sliding_window(s, model, vocabulary, 
                                         weighting = uniform,
                                         document_progress_bar = document_progress_bar,
                                         window_size = window_size)[1] 
               for s in tqdm(raw)]
metrics.append({"window":"sliding", 
                "decoding":"greedy", 
                "weighting":"uniform",
                "window_size":window_size,
                "inference_seconds":t() - start,
                "cer_before":old,
                "cer_after":levenshtein(gs, corrections).cer.mean()})

  0%|          | 0/592 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jarobyte/envs/guemes/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-34-5322f287cdc1>", line 6, in <module>
    for s in tqdm(raw)]
  File "<ipython-input-34-5322f287cdc1>", line 6, in <listcomp>
    for s in tqdm(raw)]
  File "/home/jarobyte/guemes/lib/ocr_correction.py", line 78, in correct_by_sliding_window
    *arcorrect)
  File "/home/jarobyte/guemes/lib/pytorch_decoding/seq2seq.py", line 460, in predict
    **kwargs)
  File "/home/jarobyte/guemes/lib/pytorch_decoding/seq2seq.py", line 53, in greedy_search
    next_probabilities = self.forward(X, Y)[:, -1].log_softmax(-1)
  File "/home/jarobyte/guemes/lib/pytorch_decoding/seq2seq.py", line 855, in forward
    tgt_mask = mask)
  File "/home/jarobyte/envs/guemes/lib/python3.7/site-packages/torch/nn/modules/transformer.py", line 126, in forward
    memory_key_padding_mask=

TypeError: object of type 'NoneType' has no len()

In [ ]:
start = t()
corrections = [correct_by_sliding_window(s, model, vocabulary, 
                                         decoding_method = "beam_search", 
                                         weighting = uniform, 
                                         document_progress_bar = document_progress_bar,
                                         window_size = window_size)[1] 
               for s in tqdm(raw)]
metrics.append({"window":"sliding", 
                "decoding":"beam", 
                "weighting":"uniform",
                "window_size":window_size,
                "inference_seconds":t() - start,
                "cer_before":old,
                "cer_after":levenshtein(gs, corrections).cer.mean()})

  0%|          | 0/592 [00:00<?, ?it/s]

In [32]:
pd.DataFrame(metrics).assign(improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before))

,window,decoding,window_size,inference_seconds,cer_before,cer_after,improvement
0,disjoint,greedy,100,136.404224,9.403462,18.963715,-101.667370
1,disjoint,greedy,50,258.185009,9.403462,25.778392,-174.137245
2,disjoint,beam,100,467.441658,9.403462,19.278679,-105.016814
3,disjoint,beam,50,576.490932,9.403462,25.907503,-175.510255
